# Earth's Position and Velocity

## Defining the Purpose

Spice is a well known API written in C, used for Astronomical calculations. It works by referencing recorded Kernels by bodies like NASA's JPL (Ref. notes/kernels.md) 

This notebook is my introduction to Spice and Space Sciene with Python in general by walking down through the calculations of Earth and Sun parameters

---

Importing Required Libraries

In [61]:
import spiceypy
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt

Loading the Spice Kernels

In [62]:
spiceypy.furnsh(
    [
        '../../kernels/lsk/naif0012.tls',
        '../../kernels/pck/gm_de431.tpc',
        '../../kernels/pck/pck00010.tpc',
        '../../kernels/spk/de432s.bsp'
    ]
)

Constructing a Datetime Object that Returns the Current Date-Time

In [63]:
current_date_time = datetime.datetime.today()

Formatting to the UTC Timestamp as it is better supported by Spicepy for Ephimeral Conversion

In [64]:
current_date_time = current_date_time.strftime(r"%Y-%m-%dT00:00:00")

print(f"UTC Timestamp: {current_date_time}")

UTC Timestamp: 2025-04-30T00:00:00


**Converting to Ephimeris Timestamp:**

UTC Timestamps provide valid calculations for measurements on Earth. However, measuring stellar objects from an observer's point of view requires Ephimeris Timestamps as it adjusts for relativistic effects and astronomical measurements. Emphimeris Timestamps measure the time in seconds for the current date since 12AM.

In [65]:
ephimeris_date_time = spiceypy.utc2et(current_date_time)
print(f"Ephimeris Timestamp: {ephimeris_date_time}")

Ephimeris Timestamp: 799243269.1854932


Computing the State of Earth at the Current Ephimeral Timestamp

In [66]:
# spkgeo returns the state vector (x, y, z) cartesian system (first three of ndarray),
# the tangential velocities in each of these directions (last three of ndarray)
# and the light time

earth_state_wrt_sun, earth_sun_light_time = spiceypy.spkgeo(

    targ = 399, # 399 targ denotes Earth by Spice
    et = ephimeris_date_time, # Reference Time
    ref = "ECLIPJ2000", # Plane of reference with respect to Earth (ref. notes/reference_planes.md)
    obs = 10 # As observed from Sun
)

print(f"State Vector: {earth_state_wrt_sun}")

print(f"Light Time: {earth_sun_light_time}")
light_time_in_minutes = earth_sun_light_time / 60.0
print(f"Light Time in Minutes: {light_time_in_minutes}")

State Vector: [-1.16171636e+08 -9.59668910e+07  5.96730297e+03  1.85005015e+01
 -2.30826193e+01  9.00247491e-04]
Light Time: 502.62578926939926
Light Time in Minutes: 8.377096487823321


Representation of State Vector (ECLIPJ2000)
![State Vector](../../public/state-vector.png)

*A Note on ECLIPJ2000 and Vernal Equinox*

ECLIPJ2000 Plane of Reference uses the right hand rule to point the Cartesian Axes in a way such that the X Axis points to the Vernal Equinox, Y Axis points to the Orbit and Z Axis towards the observer, as viewing the plane from the top

The vernal equinox, also known as the spring equinox, marks the beginning of spring in the Northern Hemisphere and occurs when the Sun crosses the celestial equator moving northward, making day and night approximately equal in length. On this day, the state vector if normalized, represents (x, y, z) as (1, 0, 0)

Calculating Euclidean distance from Cartesian Position

In [67]:
earth_sun_distance = math.sqrt(

    earth_state_wrt_sun[0] ** 2.0 
    + earth_state_wrt_sun[1] ** 2.0
    + earth_state_wrt_sun[2] ** 2.0
)

print(f"Distance of Earth from Sun is {earth_sun_distance} Kms")

Distance of Earth from Sun is 150683420.81926322 Kms


Converting to Astronomical Units (1 AU is the mean distance between Earth and the Sun)

In [68]:
distance_in_au = spiceypy.convrt(earth_sun_distance, "km", "au")

print(f"Distance in Astronomical Units is {distance_in_au} AU")

Distance in Astronomical Units is 1.0072564549289447 AU


Time taken by Sunlight to reach Earth

In [69]:
print(f"Light Time in Seconds: {earth_sun_light_time}")
light_time_in_minutes = earth_sun_light_time / 60.0
print(f"Light Time in Minutes: {light_time_in_minutes}")

Light Time in Seconds: 502.62578926939926
Light Time in Minutes: 8.377096487823321


**Computing Angular and Tangential Velocities**

Converting the state vector to np array for ease in calculations

In [70]:
earth_state_wrt_sun = np.array(earth_state_wrt_sun)

Calculating the distance between Earth and the Sun using normalization

In [71]:
earth_sun_distance = np.linalg.norm(earth_state_wrt_sun[:3])

print(f"Distance between Earth and Sun is {earth_sun_distance} Kms")

Distance between Earth and Sun is 150683420.81926322 Kms


Calculating the Orbital Velocity of Earth - It is the magnitude of velocities of Earth in each Cartesian Direction

In [72]:
orbital_velocity = np.linalg.norm(earth_state_wrt_sun[3:])

print(f"The Orbital Velocity of Earth is {orbital_velocity} Km/s")

The Orbital Velocity of Earth is 29.581681309011476 Km/s


Calculating Gravitation x Mass as a factor to compute the actual Orbital Velocity for verification

$$
OrbitalVelocity = \sqrt{GM/R}
$$

In [73]:
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item="GM", maxn=1)

Verifying the Orbital Velocity computed

In [74]:
verify_orbital_velocity = lambda gm, r: np.sqrt(gm/r)

print(f"Computed Velocity: {orbital_velocity} Km/s")
print(f"Verified Orbital Velocity: {verify_orbital_velocity(GM_SUN[0], earth_sun_distance)} Km/s")

Computed Velocity: 29.581681309011476 Km/s
Verified Orbital Velocity: 29.677210802063204 Km/s


Computing the Angular Distance and Actual Distance (Since Vernal Equinox)

![Angular and Actual Distance](../../public/distance.png)
![Calculation of Distances](../../public/calculate_distance.png)

In [75]:
earth_position_wrt_sun = earth_state_wrt_sun[:3]
earth_state_wrt_sun_normalized = earth_position_wrt_sun / earth_sun_distance

# Actual normalized state on vernal equinox:
earth_state_wrt_sun_normed_autumn = [1.0, 0.0, 0.0]

angular_distance = np.degrees(

    np.arccos(np.dot(

        earth_state_wrt_sun_normalized, earth_state_wrt_sun_normed_autumn
    ))
)

print(f"Angular distance covered by Earth since last Vernal Equinox is {angular_distance} deg")

Angular distance covered by Earth since last Vernal Equinox is 140.44062001652645 deg


In [76]:
actual_distance = np.radians(angular_distance) * earth_sun_distance
print(f"Actual distance covered by Earth since Vernal Equinox (Approx): {actual_distance} Km")

Actual distance covered by Earth since Vernal Equinox (Approx): 369347851.2014409 Km


*A note on this approximation:*

Since the mean distance between Sun and Earth changes with time due to the ellipsical nature of Orbit, it is impossible to accurately compute the actual arc length because of variations in 'r' in the formula:

$$
ArcLength = r (Km) * \theta
$$